In [1]:
import cv2
import time
import src.viztools as vv
from __future__ import print_function

 ### TODO

In [ ]:
reload(vv)

In [ ]:
# general parameters
USE_CAM = 0
DISP_FULL_SCREEN = 0
LOOP = True    # loop through videos

# select video source
if USE_CAM:
    cap = cv2.VideoCapture(0)
else:
    cap = cv2.VideoCapture(0)
#     cap = cv2.VideoCapture('/media/data/Dropbox/Git/vid-viz/data/tunnel_01.mp4')
#     cap = cv2.VideoCapture('/home/mattw/Videos/2016-12-18 23-15-59.mp4')
    
# get video properties
# target_fps = cap.get(cv2.CAP_PROP_FPS)
target_fps = 20.0
tot_fr_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

# initialize key handler
effect_index = None
post_process = False
key_list = [False for i in range(256)]

# initialize processing objects
effects = [vv.Threshold(),
           vv.Alien(),
           vv.RGBWalk()]
num_effects = len(effects)
border = vv.Border()

fr_count = 0
while(cap.isOpened()):

    time_pre = time.time()
    
    # get frame and relevant info
    ret, frame = cap.read()
    
    if USE_CAM:
        # crop image to have same aspect ratio as monitor
        frame = frame[60:-60,:,:]

    # get keyboard input
    key = cv2.waitKey(1) & 0xFF
    key_list[key] = True
    if key == 27:
        # escape key
        break
    elif key == ord('q'):
        # quit current effect
        effect_index = None
    elif key == ord('`'):
        post_process = not post_process
        
    if effect_index is None:
        for num in range(10):
            if key == ord(str(num)):
                effect_index = num
        
    # process frame
    if effect_index is not None and effect_index < num_effects:
        frame = effects[effect_index].process(frame, key_list)
    
    # apply borders last to save processing time
    if post_process:
        frame = border.process(frame, key_list)

    # display frame
    if DISP_FULL_SCREEN:
        cv2.namedWindow('frame', cv2.WND_PROP_FULLSCREEN)
        cv2.setWindowProperty('frame', cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)
        cv2.imshow('frame', frame)
    else:
        cv2.imshow('frame', frame)
        if fr_count == 0:
            cv2.moveWindow('frame', 0, 0)
    
    # reset video
    fr_count += 1
    if fr_count == tot_fr_count and LOOP:
        # reset frame postion to zero
        cap.set(cv2.CAP_PROP_POS_FRAMES,1)
        fr_count = 1
    
    # calculate, limit and output fps
    time_tot = time.time() - time_pre
    if time_tot < 1/target_fps:
        time.sleep(1/target_fps - time_tot)
    time_tot = time.time() - time_pre
    print('\r%03i fps' % (1.0 / time_tot), end='')
    
cap.release()
cv2.destroyWindow('frame')

019 fps